# Genetic Algorithms
A genetic algorithm (GA) is a population‑based search method inspired by evolution and natural selection. It keeps a set of candidate solutions and iteratively updates them using selection, crossover, and mutation, all guided by a fitness function that encodes how good each solution is.

Each candidate solution is encoded as a chromosome, usually a fixed‑length vector (for example a bitstring or a vector of real‑valued parameters). A population is just a collection of such chromosomes, and its size controls both how much of the search space you can explore and how expensive each generation is to evaluate.

Genetic algorithms show up in many application domains, including engineering design and scheduling, automatic program synthesis, machine learning model and hyperparameter optimization, and the calibration of scientific models in areas like economics and biology.

---

## Algorithm Outline

1. **Initialize a population of \(N\) random individuals**  
   The algorithm starts with no prior knowledge, so it samples \(N\) chromosomes at random from the search space.  
   Each chromosome is a candidate solution; the initial population is usually diverse and mostly low‑quality, but it covers different regions of the space.

2. **Evaluate the fitness of all \(N\) individuals**  
   For each chromosome, the fitness function computes a scalar score that measures how good that candidate solution is for the specific problem.  
   Genetic algorithms are typically used as maximizers: higher fitness means a better solution.

3. **Select pairs of individuals biased by fitness (survival of the fittest)**  
   The next generation is produced by letting fitter individuals reproduce more often.  
   A selection mechanism (e.g., tournament or roulette‑wheel) repeatedly chooses parents, so that individuals with higher fitness have a higher probability of being selected, while still allowing some randomness.

4. **Apply crossover to selected parent pairs to generate offspring**  
   Once parents are selected, crossover recombines their chromosomes. In one‑point crossover, for example, a random cut point is drawn and the parents exchange their tails, producing two children.  
   This operation mixes genetic material from different individuals and can combine useful building blocks (good partial solutions).

5. **Apply mutation to the offspring by randomly changing some genes**  
   After crossover, each offspring undergoes mutation, which introduces small random perturbations.  
   For bitstrings, this is often realized as bit‑flip mutation, where each gene is flipped with a small probability. Mutation helps maintain diversity and can enable the algorithm to escape local optima.

6. **Form the new population (with optional elitism)**  
   Offspring are generated until \(N\) new individuals have been created, forming the next generation.  
   Many GA variants use elitism, i.e., they copy one or more of the best individuals from the previous generation into the new population to ensure that the best solutions found so far are not lost.

7. **Check the termination criterion and repeat if needed**  
   After building the new population, the algorithm evaluates the fitness of all individuals again.  
   If the termination criterion is satisfied (e.g., a maximum number of generations, or a satisfactory fitness value), the algorithm returns the best individual found as the final solution; otherwise, it goes back to step 2 and starts a new generation.

---

## Limitations
Genetic algorithms are heuristic search methods, so they generally do not guarantee convergence to a global optimum. In practice, the search moves locally in the space of populations and is guided by fitness values, which means a GA can get stuck in local optima or flat regions, especially when diversity is low or the fitness landscape is highly multimodal. For most realistic problems, the aim is therefore not to prove global optimality, but to find a reasonably good solution within a limited computational budget.

Two common issues in genetic algorithms are premature convergence and loss of diversity.

- **Premature convergence**: the population settles too quickly around a suboptimal solution, and the best or average fitness stops improving over generations. A flat fitness curve on its own is not a good stopping rule, because it might just mean the GA is stuck in a bad local optimum rather than having really “finished” the search.

- **Loss of diversity**: individuals become very similar to each other, so crossover keeps generating almost identical offspring and the algorithm stops exploring new areas of the search space. You can often spot this when most chromosomes look alike or children barely differ from their parents.

## Hyperparameters 
The behavior of a GA is mostly driven by its hyperparameters: population size, how selection is done, crossover and mutation rates, and how you encode candidate solutions. Population size and selection pressure decide how much you explore versus exploit, crossover and mutation decide how aggressively you search new regions, and the encoding (binary, integer, real‑valued, etc.) plus the chosen operators should fit the structure of the problem you are trying to solve.

Since performance depends so much on these choices, a practical way to compare two GA setups is to run both several times on the same problem, plot the average best fitness over generations, and, if needed, look at the distribution of the final best fitness values (for example with boxplots) to see which setup tends to produce better solutions more reliably.

---

# My implementation

In this notebook I’ll work with a single, flexible `GeneticAlgorithm` class written in Python. It follows the usual GA loop we’ve just seen, but keeps most of the details configurable so I can reuse it in different problems without rewriting the core logic.

When I create a `GeneticAlgorithm` object, I mainly decide three things:

- how long chromosomes are (`chromosome_length`)
- how I measure the quality of a solution (`fitness_func`)
- how I want the population to evolve (population size, selection, crossover/mutation, elitism, random seed, …)

Once these are set, the class handles the whole evolution process and returns the best solution it finds, plus some basic stats over generations.

---

## Design
Here, each chromosome is just a Python list of genes (`List[Any]`). By default genes are bits (0 or 1), but since everything is typed as `Any` I can switch to integers, floats, characters, etc., as long as the fitness function knows what to do with them.

The main constructor arguments are:

- `chromosome_length`: number of genes per individual.
- `fitness_func: List[Any] -> float`: takes one individual and returns a fitness score (higher is better).
- `population_size`: how many individuals I keep in each generation (default 100).
- `crossover_prob`, `mutation_prob`: how often I apply crossover and how likely each gene is to mutate.
- `selection_strategy`: `"tournament"` (default) or `"roulette"`.
- `tournament_size`: group size for tournament selection.
- `crossover_operator`, `mutation_operator`: optional custom operators if I don’t want the defaults.
- `gene_initializer`: function that samples a single gene (default: random bit 0/1).
- `elitism`: whether I keep the best individual across generations.
- `random_state`: optional seed to make runs reproducible.

Internally, the class keeps:

- `population`: the current list of individuals.
- `fitness_values`: fitness scores for the current population.
- `best_individual_`, `best_fitness_`: the best solution seen so far.
- `history_`: a list of dictionaries with per‑generation stats (best and mean fitness).

---

## Population Initialization and Evaluation

To start, `initialize_population` builds the first population by calling `gene_initializer` for each gene of each individual. Changing this initializer is enough to move from bitstrings to other encodings without touching the rest.

Then `evaluate_population` runs `fitness_func` on every individual, stores the fitness values, and updates the current best solution. I call this once before the loop and then after each generation so I can track progress over time.

---

## Selection, Crossover, and Mutation

For parent selection I support two strategies:

- **Tournament selection** (`_select_parent_tournament`): I sample `tournament_size` random individuals and pick the one with the highest fitness.
- **Roulette‑wheel selection** (`_select_parent_roulette`): I build probabilities proportional to fitness (after shifting if there are negative values) and sample according to those. If the total fitness is zero, I fall back to a uniform random choice.

The method `select_parents` just wraps this and calls the right strategy based on `selection_strategy`.

For variation I provide default operators but allow overrides:

- **One‑point crossover** (`_one_point_crossover`): with probability `crossover_prob` I pick a cut point along the chromosome and swap the tails of the two parents to get two children; otherwise I simply copy the parents.
- **Bit‑flip mutation** (`_bit_flip_mutation`): I go through each gene and flip it with probability `mutation_prob` (assuming binary genes). For other encodings I can pass a different `mutation_operator` when I build the GA.

---

## Generational Update and Elitism

The method `step` builds the next generation from the current one:

- If `elitism` is on, I first find the best individual in the current population and keep a copy as `elite`.
- Then, until `new_population` reaches `population_size`:
  - I select two parents with `select_parents`.
  - I apply the crossover operator to get two children.
  - I mutate both children.
  - I add them to `new_population` (checking not to exceed the desired size).

At the end, if elitism is enabled, I overwrite a random position in the new population with the `elite` individual. This guarantees that the best solution I’ve seen so far is never lost just because of unlucky crossover or mutation.

Finally, I replace the old population with the new one.

---

## Running the Algorithm and Tracking History

The main method I use is `run`:

- It initializes the population and evaluates it once.
- For `n_generations` steps it:
  - computes best and mean fitness and appends them to `history_`,
  - prints a short log line if `verbose=True`,
  - calls an optional `callback(gen, population, fitness_values)` if I pass one,
  - calls `step()` to produce the next generation,
  - re‑evaluates the population.

At the very end it stores one last snapshot in `history_` and returns:

```python
best_individual_, best_fitness_, history_
```

This makes it easy both to retrieve the final solution and to plot how the GA behaved.

In [1]:
import random
import numpy as np
from typing import Callable, List, Tuple, Optional, Any, Dict

class GeneticAlgorithm:

    def __init__(
        self,
        chromosome_length: int,
        fitness_func: Callable[[List[Any]], float],
        population_size: int = 100,
        crossover_prob: float = 0.8,
        mutation_prob: float = 0.01,
        selection_strategy: str = "tournament",
        tournament_size: int = 3,
        crossover_operator: Optional[Callable[[List[Any], List[Any]], Tuple[List[Any], List[Any]]]] = None,
        mutation_operator: Optional[Callable[[List[Any], float], List[Any]]] = None,
        gene_initializer: Optional[Callable[[], Any]] = None,
        elitism: bool = True,
        random_state: Optional[int] = None,
    ):
        self.chromosome_length = chromosome_length
        self.fitness_func = fitness_func
        self.population_size = population_size
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.selection_strategy = selection_strategy
        self.tournament_size = tournament_size
        self.crossover_operator = crossover_operator
        self.mutation_operator = mutation_operator
        self.gene_initializer = gene_initializer
        self.elitism = elitism

        self.random_state = random_state
        if random_state is not None:
            random.seed(random_state)
            np.random.seed(random_state)

        
        self.population: List[List[Any]] = []
        self.fitness_values: List[float] = []
        self.best_individual_: Optional[List[Any]] = None
        self.best_fitness_: Optional[float] = None
        self.history_: List[Dict[str, float]] = []

        # Default
        if self.gene_initializer is None:
            # default: bitstring 0/1
            self.gene_initializer = lambda: random.randint(0, 1)

        if self.crossover_operator is None:
            self.crossover_operator = self._one_point_crossover

        if self.mutation_operator is None:
            self.mutation_operator = self._bit_flip_mutation


    # Init
    def initialize_population(self):
        self.population = [
            [self.gene_initializer() for _ in range(self.chromosome_length)]
            for _ in range(self.population_size)
        ]

    def evaluate_population(self):
        self.fitness_values = [self.fitness_func(ind) for ind in self.population]

        gen_best_idx = int(np.argmax(self.fitness_values))
        gen_best_fit = self.fitness_values[gen_best_idx]
        gen_best_ind = self.population[gen_best_idx]

        if (self.best_fitness_ is None) or (gen_best_fit > self.best_fitness_):
            self.best_fitness_ = gen_best_fit
            self.best_individual_ = gen_best_ind.copy()

    # Selection
    def _select_parent_tournament(self) -> List[Any]:
        indices = np.random.choice(
            len(self.population), size=self.tournament_size, replace=False
        )
        best_idx = indices[0]
        best_fit = self.fitness_values[best_idx]
        for idx in indices[1:]:
            if self.fitness_values[idx] > best_fit:
                best_fit = self.fitness_values[idx]
                best_idx = idx
        return self.population[best_idx]

    def _select_parent_roulette(self) -> List[Any]:
        fitness_array = np.array(self.fitness_values, dtype=float)
        min_fit = fitness_array.min()
        if min_fit < 0:
            fitness_array -= min_fit 
        total_fit = fitness_array.sum()
        if total_fit == 0:
            idx = np.random.randint(len(self.population))
            return self.population[idx]

        probs = fitness_array / total_fit
        idx = np.random.choice(len(self.population), p=probs)
        return self.population[idx]

    def select_parents(self) -> Tuple[List[Any], List[Any]]:
        if self.selection_strategy == "roulette":
            p1 = self._select_parent_roulette()
            p2 = self._select_parent_roulette()
        else:
            # default: tournament
            p1 = self._select_parent_tournament()
            p2 = self._select_parent_tournament()
        return p1, p2

    # Crossover and mutation
    def _one_point_crossover(
        self,
        parent1: List[Any],
        parent2: List[Any],
    ) -> Tuple[List[Any], List[Any]]:
        if len(parent1) != len(parent2):
            raise ValueError("Chromosomes must have the same length for the crossover.")

        if random.random() > self.crossover_prob:
            return parent1.copy(), parent2.copy()

        point = random.randint(1, len(parent1) - 1) 
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2

    def _bit_flip_mutation(
        self,
        individual: List[Any],
        mutation_prob: float,
    ) -> List[Any]:
        mutated = []
        for gene in individual:
            if random.random() < mutation_prob:
                mutated.append(1 - gene)  
            else:
                mutated.append(gene)
        return mutated

    # Generation
    def step(self):
        new_population: List[List[Any]] = []

        # Elitism
        elite = None
        if self.elitism:
            best_idx = int(np.argmax(self.fitness_values))
            elite = self.population[best_idx].copy()

        # Generate new population
        while len(new_population) < self.population_size:
            parent1, parent2 = self.select_parents()
            child1, child2 = self.crossover_operator(parent1, parent2)

            child1 = self.mutation_operator(child1, self.mutation_prob)
            child2 = self.mutation_operator(child2, self.mutation_prob)

            new_population.append(child1)
            if len(new_population) < self.population_size:
                new_population.append(child2)

        if self.elitism and elite is not None:
            replace_idx = random.randint(0, self.population_size - 1)
            new_population[replace_idx] = elite

        self.population = new_population

    # Run
    def run(
        self,
        n_generations: int = 100,
        callback: Optional[Callable[[int, List[List[Any]], List[float]], None]] = None,
        verbose: bool = False,
    ) -> Tuple[List[Any], float, List[Dict[str, float]]]:
        """
        Run the algo for n_generations.
        Return:
            best_individual, best_fitness, history
        """
        self.initialize_population()
        self.evaluate_population()

        self.history_ = []
        for gen in range(n_generations):
            best_fit = float(np.max(self.fitness_values))
            mean_fit = float(np.mean(self.fitness_values))

            self.history_.append(
                {
                    "generation": gen,
                    "best_fitness": best_fit,
                    "mean_fitness": mean_fit,
                }
            )

            if verbose:
                print(
                    f"Gen {gen:4d} | "
                    f"best: {best_fit:.4f} | mean: {mean_fit:.4f}"
                )

            if callback is not None:
                callback(gen, self.population, self.fitness_values)

            self.step()
            self.evaluate_population()

        best_fit = float(np.max(self.fitness_values))
        mean_fit = float(np.mean(self.fitness_values))

        self.history_.append(
            {
                "generation": n_generations,
                "best_fitness": best_fit,
                "mean_fitness": mean_fit,
            }
        )

        if verbose:
            print(
                f"Gen {n_generations:4d} | "
                f"best: {best_fit:.4f} | mean: {mean_fit:.4f}"
            )

        return self.best_individual_, self.best_fitness_, self.history_

## Example 1 – Bitstring optimization (OneMax)

As a first example I will apply the `GeneticAlgorithm` class to a very simple binary optimization problem. The goal is to evolve a bitstring of fixed length so that it contains as many ones as possible. This problem is often called *OneMax*: given a chromosome $\mathbf{x} = (x_1, \dots, x_L)$ with $x_i \in \{0, 1\}$, the fitness is just the sum of its bits
$$
\text{fitness}(\mathbf{x}) = \sum_{i=1}^L x_i.
$$

In this setting:

- the chromosome is a binary vector of length \(L\);
- the default gene initializer (random 0/1) is exactly what we need;
- the fitness function simply counts the number of ones in the chromosome.

The global optimum is the all‑ones string, so it is easy to check whether the GA is behaving as expected and to visualize how quickly it converges to the optimum.


In [2]:
# Fitness function: count the number of ones
def onemax_fitness(individual: List[Any]) -> float:
    # individual is a list of 0/1 values
    return float(sum(individual))

chromosome_length = 40
population_size = 100
n_generations = 20

ga_onemax = GeneticAlgorithm(
    chromosome_length=chromosome_length,
    fitness_func=onemax_fitness,
    population_size=population_size,
    crossover_prob=0.8,
    mutation_prob=0.01,
    selection_strategy="tournament",
    tournament_size=3,
    elitism=True,
    random_state=42,
)

def onemax_callback(gen: int, population, fitness_values):
    best_idx = int(np.argmax(fitness_values))
    best = population[best_idx]
    best_fit = fitness_values[best_idx]
    print(f"Gen {gen:3d} | best fitness: {best_fit:.0f} | best: {best}")

best_ind, best_fit, history = ga_onemax.run(
    n_generations=n_generations,
    verbose=False,              
    callback=onemax_callback,  
)

print("\nBest fitness:", best_fit)
print("Best individual:", best_ind)

Gen   0 | best fitness: 27 | best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0]
Gen   1 | best fitness: 28 | best: [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
Gen   2 | best fitness: 31 | best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0]
Gen   3 | best fitness: 33 | best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0]
Gen   4 | best fitness: 33 | best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
Gen   5 | best fitness: 35 | best: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
Gen   6 | best fitness: 37 | best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Example 2 – Evolving a Text String

In the second example I use the `GeneticAlgorithm` class to evolve a target sentence character by character. Each chromosome is a sequence of characters, and the GA tries to make this sequence match a fixed target string.

Here:
- The chromosome is a list of characters with the same length as the target sentence.
- The gene initializer samples random characters from a fixed alphabet (letters, spaces, punctuation).
- The fitness function compares the candidate string to the target and returns the fraction of characters that are correct in the correct position.
- Mutation randomly replaces characters with new ones from the alphabet, while crossover works as before, recombining substrings from two parents.

Because the fitness is normalized between 0 and 1, it is easy to monitor how close the current best individual is to the target sentence. By printing the best string every few generations, we can watch the GA gradually turn random noise into a readable sentence.

In [3]:
target = "Hi, my name is Tommaso and I'm a data science student at Ca'Foscari university."
alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ,.'"

def fitness_string(individual: List[Any]) -> float:
    s = "".join(individual)
    correct = sum(c1 == c2 for c1, c2 in zip(s, target))
    return correct / len(target)

def init_gene_char() -> str:
    return random.choice(alphabet)

def char_mutation(individual: List[Any], mutation_prob: float) -> List[Any]:
    mutated = []
    for gene in individual:
        if random.random() < mutation_prob:
            mutated.append(random.choice(alphabet))
        else:
            mutated.append(gene)
    return mutated

ga_str = GeneticAlgorithm(
    chromosome_length=len(target),
    fitness_func=fitness_string,
    population_size=80,
    crossover_prob=0.8,
    mutation_prob=0.02,
    selection_strategy="tournament",
    elitism=True,
    gene_initializer=init_gene_char,
    mutation_operator=char_mutation,
    random_state=123,
)

def print_best_string(gen: int, population, fitness_values):
    if gen % 10 == 0:
        best_idx = int(np.argmax(fitness_values))
        best = population[best_idx]
        best_str = "".join(best)
        best_fit = fitness_values[best_idx]
        print(f"[{gen:3d}] '{best_str}'")

best_individual_str, best_fitness_str, history_str = ga_str.run(
    n_generations=500,
    callback=print_best_string,
    verbose=False,
)

print("\n=== Results ===")
print("Target:", target)
print("Best  :", "".join(best_individual_str))
print("Fitness:", best_fitness_str)

[  0] 'xs,jB,VQ,KemWD,kb JjhjlYJ.JI,HadfpDEQ X,HGwlSA'Fgb.SMqnOyMOvVGKbIllmsYlFbz.iuNF'
[ 10] 'xs,Fvyq.lnemAi,bb JdhjlnJ.JI,H FBaGCS'snYLO,eJaDZvdmWkasGca.ToCBk sttiXyUpkVEy.'
[ 20] 'Hs, byqnPYemWO,Nb JjhBlYJ'JI,H FBaGCS'snYeO,eJaDZv.nWaasGca.ToCBkS'tkCWveTkVEy.'
[ 30] 'Hs, xyqnPYeaiYTZK JjvBlaJ'JI,H FBaGRS'snNeO,eusjhbFnbkashNa.To'Bkr'tkCEveTkVEy.'
[ 40] 'Hs, byqnPYeaiYTFo JjaqlaJ'JI,H FBacRS'saYeg,e stZdfnthaAhNa.ToCBkrktkCWverkVqy.'
[ 50] 'Hs, MyqnaYeaiYTFo JjvBiaJ'JIVH FTayRz'sniegSe studAnbiashCa.ToCBkr'tkcWverkiJy.'
[ 60] 'Hsf MyonaYeaisTFo JjaolanWJI'H aBayRzusnienSe sTudenbiashCa.ToXUkr'DkCWXersiGy.'
[ 70] 'HR, MyonaYeaisTko Jjmolan JI'H atacRS'sbienSe studenbias Ca.ToCUkr'tkcWversiNy.'
[ 80] 'Hu, MyonaYeaisTRo JjaolanaJI'H atacRS'sbienSe studenbvas CauFoC'Yr'tuciversiGy.'
[ 90] 'Hl,AMyqnaAeAisJRo JYaolana I'H atacRS'sbienSe studentBas CauFoC'YrQtuniversiIy.'
[100] 'HM, MyonaseAistRomJYaolana I'H attaRS'sbienSe studentBas CauFoCoYr'tuniversiIy.'
[110] 'Hv, MyYnameAis.domJYToqan

## Example 3 – Feature selection on the Wine dataset

In the last example I use the `GeneticAlgorithm` class for a small but realistic feature selection task. The dataset is the classic Wine dataset from scikit‑learn: each sample is a wine characterized by 13 chemical analysis features and a class label indicating its cultivar.

Here, each chromosome is a binary mask over the 13 input features. A gene equal to 1 means that the corresponding feature is used, while 0 means that the feature is dropped. The fitness of an individual is defined as the cross‑validated classification accuracy of a Logistic Regression model trained only on the selected features, minus a small penalty proportional to the fraction of features used. This encourages the GA to find compact subsets of features that still give good predictive performance.

This example shows how to plug a real scikit‑learn model into the fitness function and use the same GA infrastructure to search over combinatorial subsets of features instead of simple bitstrings or characters.

In [4]:
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

data = load_wine()
X = data.data
y = data.target

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Feature names:", data.feature_names)
print("Target classes:", data.target_names)

df = pd.DataFrame(X, columns=data.feature_names)
df["target"] = y
df.head()

X shape: (178, 13)
y shape: (178,)
Feature names: ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
Target classes: ['class_0' 'class_1' 'class_2']


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [5]:
n_features = X.shape[1]

# Build a simple pipeline: scaling + logistic regression
base_model = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(max_iter=1000))
    ]
)

# Fitness function: CV accuracy - penalty for using many features
def wine_feature_fitness(individual: List[Any]) -> float:
    mask = np.array(individual, dtype=bool)

    # Avoid empty or degenerate masks
    if mask.sum() == 0:
        return 0.0

    X_sel = X[:, mask]

    scores = cross_val_score(
        base_model,
        X_sel,
        y,
        cv=3,
        scoring="accuracy",
        n_jobs=-1,
    )
    mean_score = float(scores.mean())

    # Penalty term: fraction of selected features
    feature_fraction = mask.sum() / n_features
    penalty = 0.05 * feature_fraction 

    return mean_score - penalty

# GA setup
chromosome_length = n_features
population_size = 100
n_generations = 10

ga_wine = GeneticAlgorithm(
    chromosome_length=chromosome_length,
    fitness_func=wine_feature_fitness,
    population_size=population_size,
    crossover_prob=0.8,
    mutation_prob=0.05,
    selection_strategy="tournament",
    tournament_size=3,
    elitism=True,
    random_state=123,
)

from typing import List, Any
import numpy as np

def wine_callback(gen: int, population: List[List[Any]], fitness_values: List[float]):
    best_idx = int(np.argmax(fitness_values))
    best = population[best_idx]
    best_fit = fitness_values[best_idx]
    mask = np.array(best, dtype=bool)
    n_sel = mask.sum()
    selected_features = np.array(data.feature_names)[mask]
    print(f"[{gen:3d}] fit={best_fit:.3f} | sel={n_sel}/{n_features} | features={', '.join(selected_features)}\n")

best_mask, best_fit, history_wine = ga_wine.run(
    n_generations=n_generations,
    callback=wine_callback,  
    verbose=False,
)


best_mask = np.array(best_mask, dtype=bool)
selected = np.array(data.feature_names)[best_mask]

print("\n=== Wine feature selection results ===")
print("Best fitness:", best_fit)
print("Selected features:", best_mask.sum(), "/", n_features, "(", ", ".join(map(str, selected)), ")")

# Compare accuracy with all features vs selected subset
scores_all = cross_val_score(base_model, X, y, cv=3, scoring="accuracy", n_jobs=-1)
scores_sel = cross_val_score(base_model, X[:, best_mask], y, cv=3, scoring="accuracy", n_jobs=-1)

print(f"\nAccuracy with all features: {scores_all.mean():.3f}")
print(f"Accuracy with GA-selected features: {scores_sel.mean():.3f}")

[  0] fit=0.943 | sel=9/13 | features=alcohol, ash, alcalinity_of_ash, magnesium, flavanoids, nonflavanoid_phenols, color_intensity, od280/od315_of_diluted_wines, proline

[  1] fit=0.962 | sel=7/13 | features=alcohol, ash, alcalinity_of_ash, nonflavanoid_phenols, hue, od280/od315_of_diluted_wines, proline

[  2] fit=0.962 | sel=7/13 | features=alcohol, ash, alcalinity_of_ash, nonflavanoid_phenols, hue, od280/od315_of_diluted_wines, proline

[  3] fit=0.966 | sel=6/13 | features=alcohol, ash, alcalinity_of_ash, hue, od280/od315_of_diluted_wines, proline

[  4] fit=0.966 | sel=6/13 | features=alcohol, ash, alcalinity_of_ash, hue, od280/od315_of_diluted_wines, proline

[  5] fit=0.966 | sel=6/13 | features=alcohol, ash, alcalinity_of_ash, hue, od280/od315_of_diluted_wines, proline

[  6] fit=0.966 | sel=6/13 | features=alcohol, ash, alcalinity_of_ash, hue, od280/od315_of_diluted_wines, proline

[  7] fit=0.968 | sel=7/13 | features=alcohol, ash, alcalinity_of_ash, flavanoids, hue, od280/